# Nettoyage des données

## 0. Set up

### Mise en place de l'environnement

In [25]:
#Importation des Modules nécessaires
import pandas as pd
import sqlalchemy
import mysql.connector

#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys
sys.path.insert(0, "/home/apprenant/simplon_project/American_dream")


#On peut ensuite importer le dossier comme un module

from src.d00_utils.mysql_utils import mysql_connect, save_to_mysql
connect = mysql_connect()

### Importation des données depuis my sql

In [26]:
df1 = pd.read_sql("survey_1",con=connect)
dfk = pd.read_sql("survey_k",con=connect)

## 1. Selection des colonnes

J'affiche un échantillno de mes données pour analyser la pertinence des différentes colonnes

In [27]:
print(df1.head())
print(df1.columns)

   index  Survey Year           Timestamp  SalaryUSD        Country  \
0      0         2020 2020-01-04 18:50:34   115000.0  United States   
1      1         2020 2020-01-04 10:43:02   100000.0  United States   
2      2         2020 2020-01-04 09:51:46   100000.0          Spain   
3      3         2020 2020-01-04 01:08:54    70000.0  United States   
4      4         2020 2020-01-03 15:28:54   110000.0  United States   

  PostalCode       PrimaryDatabase  YearsWithThisDatabase  \
0         03  Microsoft SQL Server                     15   
1       None                 Other                      6   
2      28046  Microsoft SQL Server                      2   
3      94133  Microsoft SQL Server                      3   
4      95354                Oracle                     30   

                                      OtherDatabases    EmploymentStatus  ...  \
0  Microsoft SQL Server, MongoDB, Azure SQL DB (a...  Full time employee  ...   
1                                      MySQL

Je choisis uniquement les colonnes qui pourraient m'être utiles pour mon étude

In [28]:
df1 = df1[['index','SalaryUSD','Country','PostalCode','EmploymentStatus','JobTitle','ManageStaff','YearsWithThisTypeOfJob',
          'HowManyCompanies','OtherPeopleOnYourTeam', 'CompanyEmployeesOverall',
          'PopulationOfLargestCityWithin20Miles','EmploymentSector', 'LookingForAnotherJob', 'CareerPlansThisYear',
          'Gender', 'OtherJobDuties']]

Je fais la même chose pour la seconde table

## 2. Valeurs manquantes

### Recherche des valeurs manquantes 

In [29]:
print(df1.isnull().sum()) 
print(df1.shape)

index                                      0
SalaryUSD                                  0
Country                                    0
PostalCode                              1390
EmploymentStatus                           0
JobTitle                                   0
ManageStaff                                0
YearsWithThisTypeOfJob                     0
HowManyCompanies                           0
OtherPeopleOnYourTeam                      0
CompanyEmployeesOverall                    0
PopulationOfLargestCityWithin20Miles       0
EmploymentSector                           0
LookingForAnotherJob                       0
CareerPlansThisYear                        0
Gender                                     0
OtherJobDuties                           620
dtype: int64
(8627, 17)


Il me manque 1390 veleurs pour PostalCode et 620 pour OtherJobDUties sur un total de 8627 lignes.
Ces deux colonnes ne sont pas au centre de mon études donc ça se vaut pas le cout de supprimer toute la ligne pour elles

### traitement de Postal code

Cela ne ferait pas de sens de remplacer le code postal par un code postal arbitraire

### Traitement de OtherJobDuties

In [30]:
print(df1.OtherJobDuties.unique())
print(df1.OtherJobDuties.value_counts())

 (Production Focus - build & troubleshoot servers, HA/DR), DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Engineer, Manager'
 'Architect, DBA (Production Focus - build & troubleshoot servers, HA/DR), DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Developer: App code (C#, JS, etc)'
 'Developer: App code (C#, JS, etc), Developer: Business Intelligence (SSRS, PowerBI, etc), Developer: T-SQL, Engineer, Manager'
 'Analyst, Architect, DBA (Development Focus - tunes queries, indexes, does deployments), DBA (Production Focus - build & troubleshoot servers, HA/DR), Developer: App code (C#, JS, etc), Developer: T-SQL, Manager'
 'Analyst, Architect, DBA (Development Focus - tunes queries, indexes, does deployments), DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers), Developer: App code (C#, JS, etc), Developer: T-SQL, En

Comme il y a trop de valeur unique différente pour OtherJobDuties, je  ne peux remplacer les valeurs manquantes par une valeur standard

### Conclusion sur les valeurs manquantes

Je choisis donc de conserver l'ensemble des données dans ma base mais je note un point de vigilance quand j'aurais à utiliser les variables codePostal et OtherJobDuties

### Même travail pour la seconde base de données

## 3. Traitement des dates

In [31]:
print(df1.dtypes)

index                                     int64
SalaryUSD                               float64
Country                                  object
PostalCode                               object
EmploymentStatus                         object
JobTitle                                 object
ManageStaff                              object
YearsWithThisTypeOfJob                    int64
HowManyCompanies                         object
OtherPeopleOnYourTeam                    object
CompanyEmployeesOverall                  object
PopulationOfLargestCityWithin20Miles     object
EmploymentSector                         object
LookingForAnotherJob                     object
CareerPlansThisYear                      object
Gender                                   object
OtherJobDuties                           object
dtype: object


Aucune de mes colonnes ne correspond à une date, je n'ai donc pas à faire de traitement

## 4. Traitement des doublons

In [32]:
print(df1.duplicated().value_counts())

False    8627
dtype: int64


Aucune des colonnes de ma base n'est dupliquée, je vérifie cependant que c'est également le cas sur la colonne index

In [33]:
print(df1.duplicated('index').value_counts())

False    8627
dtype: int64


On est bon.

## 5. Consistance des données

In [34]:
x = df1["JobTitle"].value_counts()
jobs = df1.loc[df1["JobTitle"].isin(x[(x < 5)].index), "JobTitle"] = "Other"
x

DBA                                                                                                           1514
DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)    1493
DBA (Production Focus - build & troubleshoot servers, HA/DR)                                                   968
Developer: T-SQL                                                                                               826
Architect                                                                                                      660
Developer: Business Intelligence (SSRS, PowerBI, etc)                                                          610
Developer: App code (C#, JS, etc)                                                                              586
Manager                                                                                                        507
Analyst                                                                         

In [41]:
genders = df1["Gender"]
for gender in genders:
    if gender not in ["Male", "Female", 'Non-binary/third gender', 'Prefer not to say']:
        genders.replace(to_replace = gender , value = 'Prefer not to say', inplace = True)
genders

0                    Male
1                    Male
2                    Male
3                    Male
4                    Male
              ...        
8622    Prefer not to say
8623    Prefer not to say
8624    Prefer not to say
8625    Prefer not to say
8626    Prefer not to say
Name: Gender, Length: 8627, dtype: object

In [42]:
genders.unique()

array(['Male', 'Female', 'Non-binary/third gender', 'Prefer not to say'],
      dtype=object)

toutes les valeurs qui ne correspondait pas au genre (Masculin, Feminin ou Non-binaire) on été remplacé par Prefer not to say

In [44]:
salaries = df1["SalaryUSD"]
for salary in salaries : 
    if salary < 10000 : 
        salaries.replace(to_replace = salary , value = 10000 , inplace = True )
        print(salary)

101.44
1.0
57.6
9000.0
2000.0
1631.0
144.0
2200.0
8000.0
447.0
120.0
44.554
5000.0
4500.0
125.0
100.0
1556.0
0.0
135.0
150.0
75.0
9546.0
3300.0
2400.0
2900.0
3500.0
9100.0
6260.0
9360.0
7800.0
624.8
112.0
87.83
5800.0
8230.0
7500.0
900.0
58.0
1600.0
4000.0
4200.0
1967.0
56.0
9900.0
3580.0
81.0
6516.5
130.0
2246.0
700.0
38.736
126.0
5214.0
577.0
92.27
300.0
1100.0
3600.0
44.806
1200.0
3000.0
1050.0
1467.0
430.0
7473.0
8500.0
8200.0
545.0
4118.0
7650.0
6200.0
7515.0
1250.0
5500.0
4420.0
4423.0
8400.0
6700.0
3900.0
8300.0
7968.0


### Avant de partir je sauvegarde mon travail dans my sql

In [37]:
save_to_mysql(db_connect=connect,df_to_save=df1,df_name='df1_intermediate')